In [4]:
import uncertainpy as un
import chaospy as cp                       # To create distributions
import numpy as np                         # For the time array
from scipy.integrate import odeint         # To integrate our equation

import matplotlib.pyplot as plt

In [5]:
# Create the coffee cup model function
def sir_model(mp, gamma):
    # Initial  and time array
    N = 1000000  # population
    b = mp/N    # propagation rate
    dt = 0.1       # discritized time
    times = np.arange(0.0, 180, dt)
    init_vals=[N-10, 10, 0]
    info = {}

    # The equation describing the model
    def f(v, t, beta):
        x = v[0]
        y = v[1]
        z = v[2]
        dxdt = - beta*x*y
        dydt = beta*x*y - gamma*y
        dzdt = gamma*y
        return [dxdt, dydt, dzdt]

    # Solving the equation by integration
    results = odeint(f, init_vals, times, args=(b,))

    x = results[:, 0]
    info["y"] = results[:, 1]
    info["z"] = results[:, 2]

    # Return time and model output
    return times, x, info


def get_model_y(times, x, info):
    return time, info["y"]

def get_model_z(times, x, info):
    return time, info["z"]

In [7]:
model = un.Model(run=sir_model, labels=["Time", "Susceptable"])

In [8]:
mp_dist = cp.Normal(0.2, 0.01)
gamma_dist = cp.Normal(0.07, 0.005)

In [9]:
parameters = {"mp": mp_dist, "gamma":gamma_dist}

In [10]:
feature_list = [get_model_y, get_model_z]

In [11]:
UQ = un.UncertaintyQuantification(model=model, parameters=parameters, features=feature_list)

In [12]:
data = UQ.quantify(seed=10)

Calculating statistics from PCE: 100%|██████████| 3/3 [00:20<00:00,  6.99s/it]
Saving data as: data/sir_model.h5


In [13]:
data